In [7]:
#open pandas DF 
import pandas as pd
data= pd.read_csv("../Raw_Data/NCRMP_Florida_raw.csv")
print(data.columns)

Index(['time', 'latitude', 'longitude', 'REGION', 'PRIMARY_SAMPLE_UNIT',
       'STATION_NR', 'YEAR', 'MONTH', 'DAY', 'Date_UTC', 'HABITAT_CD',
       'HABITAT_TYPE', 'STRAT', 'Description', 'RUGOSITY_CD', 'WTD_RUG',
       'MAPGRID_NR', 'SUB_REGION_NAME', 'SUB_REGION_NR', 'ZONE_NAME',
       'ZONE_NR', 'MPA_NAME', 'MPA_NR', 'PROT', 'ADMIN', 'DEPTH_STRAT',
       'MIN_DEPTH', 'MAX_DEPTH', 'METERS_COMPLETED', 'SPECIES_CD',
       'SPECIES_NAME', 'N', 'JUV', 'MAX_DIAMETER', 'PERP_DIAMETER', 'HEIGHT',
       'OLD_MORT', 'RECENT_MORT', 'BLEACH_CONDITION', 'DISEASE',
       'accession_url'],
      dtype='object')


/var/folders/17/9kvd528n47j282gcln4v5zvc0000gp/T/ipykernel_2184/2430142218.py:3: DtypeWarning: Columns (1,2,25,26,27,28,33,34,35,36,37) have mixed types. Specify dtype option on import or set low_memory=False.
  data= pd.read_csv("../Raw_Data/NCRMP_Florida_raw.csv")


In [8]:
# isolating Genus for counting
data = data[["SPECIES_CD","SPECIES_NAME","DISEASE"]]
data = data.dropna(how='all').reset_index(drop=True)
data

,SPECIES_CD,SPECIES_NAME,DISEASE
0,ACR CERV,Acropora cervicornis,NaN
1,ACR CERV,Acropora cervicornis,NaN
2,ACR CERV,Acropora cervicornis,NaN
3,ACR CERV,Acropora cervicornis,NaN
4,ACR CERV,Acropora cervicornis,NaN
...,...,...,...
69403,STE INTE,Stephanocoenia intersepta,A
69404,STE INTE,Stephanocoenia intersepta,A
69405,STE INTE,Stephanocoenia intersepta,NaN
69406,STE INTE,Stephanocoenia intersepta,NaN


In [9]:
# select Disease column and setting empty cells to healthy 
data['DISEASE'][data['DISEASE'].isnull()] = "Unknown"
data

,SPECIES_CD,SPECIES_NAME,DISEASE
0,ACR CERV,Acropora cervicornis,Unknown
1,ACR CERV,Acropora cervicornis,Unknown
2,ACR CERV,Acropora cervicornis,Unknown
3,ACR CERV,Acropora cervicornis,Unknown
4,ACR CERV,Acropora cervicornis,Unknown
...,...,...,...
69403,STE INTE,Stephanocoenia intersepta,A
69404,STE INTE,Stephanocoenia intersepta,A
69405,STE INTE,Stephanocoenia intersepta,Unknown
69406,STE INTE,Stephanocoenia intersepta,Unknown


In [10]:
#Count of corals by genus and Disease column 
Grouped_data = data.groupby(['SPECIES_NAME','DISEASE']).agg(Disease_count = ("DISEASE", "count")).reset_index()

Grouped_data

,SPECIES_NAME,DISEASE,Disease_count
0,Acropora cervicornis,A,233
1,Acropora cervicornis,F,2
2,Acropora cervicornis,P,7
3,Acropora cervicornis,Unknown,1050
4,Acropora palmata,A,14
...,...,...,...
169,Stephanocoenia intersepta,P,64
170,Stephanocoenia intersepta,S,5
171,Stephanocoenia intersepta,Unknown,469
172,Undaria spp,A,2


In [11]:
from numpy import isfinite
from math import isnan
disease_df = data
unique_diseases = set(disease_df["DISEASE"].unique())

df = data
unique_species = set(df["SPECIES_NAME"].unique())
unique_species = list(unique_species)
print(unique_species)
print(unique_diseases)

disease_table = pd.DataFrame(0, index=unique_species,columns=unique_diseases)
for species_name in unique_species:
    for disease in unique_diseases: 
        disease_subtable = df.loc[(df['SPECIES_NAME'] == species_name) & (df['DISEASE'] == disease)]
        empty = bool(disease_subtable.empty)
        if not empty:
            count = disease_subtable['DISEASE'].count()
        
            disease_table.loc[species_name, disease] = count
disease_table.sort_index()

disease_table = disease_table.rename(columns ={'A':'healthy_count', 'P':'total_diseased_count', 'Unknown':'unknown_count'})
disease_table

disease_table["total_diseased_percent"] = disease_table["total_diseased_count"]/(disease_table["healthy_count"]+ disease_table["total_diseased_count"])

disease_table = disease_table.drop(columns=['F','S'])
disease_table = disease_table.rename_axis('species')
disease_table

['Favia fragum', 'Solenastrea spp', 'Porites branneri', 'Dichocoenia stokesii', 'Isophyllia sinuosa', 'Madracis spp', 'Eusmilia fastigiata', 'Mussa angulosa', 'Oculina robusta', 'Cladocora arbuscula', 'Porites furcata', 'Pseudodiploria clivosa', 'Isophyllastrea rigida', 'Madracis senaria', 'Acropora palmata', 'Mycetophyllia spp', 'Siderastrea spp', 'Agaricia agaricites', 'Montastraea cavernosa', 'Acropora cervicornis', 'Orbicella annularis', 'Porites astreoides', 'Madracis decactis', 'Orbicella faveolata', 'Manicina areolata', 'Meandrina danae', 'Siderastrea siderea', 'Agaricia lamarcki', 'Orbicella franksi', 'Mycetophyllia aliciae', 'Porites porites', 'Mycetophyllia danaana', 'Madracis auretenra', 'Agaricia humilis', 'Diploria labyrinthiformis', 'Madracis formosa', 'Agaricia grahamae', 'Siderastrea radians', 'Madracis pharensis', 'Porites spp', 'Agaricia spp', 'Helioceris cucullata', 'Stephanocoenia intersepta', 'Scolymia lacera', 'Porites divaricata', 'Dendrogyra cylindrus', 'Solenas

,total_diseased_count,healthy_count,unknown_count,total_diseased_percent
species,,,,
Favia fragum,0,11,1085,0.000000
Solenastrea spp,0,2,178,0.000000
Porites branneri,0,2,127,0.000000
Dichocoenia stokesii,1,275,1076,0.003623
Isophyllia sinuosa,0,1,282,0.000000
...,...,...,...,...
Scolymia spp,0,10,763,0.000000
Colpophyllia natans,3,348,965,0.008547
Porites colonensis,0,5,189,0.000000


In [12]:
disease_table.to_csv("../Processed_Data/NCRMP_Florida_corals.csv",index_label="species_name")